In [ ]:
#author: Julien Radoux
#Aim: this script is part of the Lifewatch IJI case study on IAS incidence

import subprocess, os, sys

#set the paths to the apps
otbbin = "/home/ubuntu/OTB-7.1.0-Linux64/bin/"
otbLWapps = "/home/ubuntu/biotopeVC/otb_appsLW/"

#get input file names from orchestrator
inRaster = sys.argv[1]
inCubeTot = sys.argv[2]
inCubeIAS = sys.argv[3]

#prepare output file name
outTot = inRaster[:-4]+ "_tot.tif"
outIAS = inRaster[:-4]+ "_totias.tif"
outIncidence = inRaster[:-4]+ "_incidence.tif"

#suffix with usefull image processing parameter. Not needed with the LookUp filter because it already includes those parameters
s = "?&gdal:co:COMPRESS=LZW&gdal:co:TILED=YES&gdal:co:BIGTIFF=YES"

#apart from the names of the input files, the parameters define the background value (0 if no observation), the column index of the "key" field (1 for the second field) and the column index of the value field (3 for the fourth field)
subprocess.call([otbLWapps + "lwLookUpAggregate", outTot, inRaster, inCubeTot, "0", "1", "3"])
subprocess.call([otbLWapps + "lwLookUpAggregate", outIAS, inRaster, inCubeIAS, "0", "1", "3"])

#combine the two outputs to compute the incidence
subprocess.call([otbbin + "otbcli_BandMath", "-il", outTot, outIAS,
                 "-out", outIncidence + s,
                 "-exp", "(im1b1<=0)?(-1):(im2b1/im1b1)"])
